In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pymongo
import requests

In [2]:
# mongoDB에 저장된 시도 목록을 얻어온다.
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017/')
starbucks = mongoConnection['starbucks']
sido = starbucks['sido']
sidoList = sido.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1})
# for s in sidoList:
    # print(s)

시도 이름을 입력받아 시도 이름에 해당되는 시도 코드를 sido 컬렉션에서 얻어온 후 크롤링하기

In [3]:
for s in sidoList:
    print(s['sido_cd'], end=' ')

01 08 02 03 04 05 06 07 09 10 11 12 13 14 15 16 17 

In [8]:
sidoName = input('구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
# sido 컬렉션에서 입력받은 시도 이름에 해당되는 도큐먼트 1건을 얻어온다.
sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()
print(sido_cd['sido_cd'])

request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', data={
    'sido_cd': sido_cd['sido_cd']
}).json()
gugunList = []
for r in request['list']:
    d = {'gugun_cd': r['gugun_cd'].strip(), 'gugun_nm': r['gugun_nm'].strip()}
    # print(d)
    gugunList.append(d)

gugun = starbucks['gugun'] # gugun 컬렉션을 만든다.
gugun.delete_many({}) # gugun 컬렉션에 저장된 모든 데이터를 제거한다.
for g in gugunList:
    gugun.insert_one(g)

구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 광주
02


In [9]:
gugunList = gugun.find({}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1})
for s in gugunList:
    print(s)

{'gugun_cd': '0201', 'gugun_nm': '광산구'}
{'gugun_cd': '0202', 'gugun_nm': '남구'}
{'gugun_cd': '0203', 'gugun_nm': '동구'}
{'gugun_cd': '0204', 'gugun_nm': '북구'}
{'gugun_cd': '0205', 'gugun_nm': '서구'}


시도 이름에 따른 전체 구군을 컬렉션에 도큐먼트로 저장하기

In [6]:
# starbucks 데이터베이스의 sido 컬렉션에 저장된 도큐먼트 개수만큼 반복하여 모든 구군을 gugun 컬렉션에 도큐먼트로 저장한다.
gugunList = []
sidoList = sido.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1})
for s in sidoList:
    request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', data={
        'sido_cd': s['sido_cd']
    }).json()
    for r in request['list']:
        # 세종 None 처리
        if r['gugun_nm'] is None:
            r['gugun_nm'] = ''
        d = {'sido_cd': s['sido_cd'].strip(), 'sido_nm': s['sido_nm'].strip(), 'gugun_cd': r['gugun_cd'].strip(),
            'gugun_nm': r['gugun_nm'].strip()}
        gugunList.append(d)

gugunAll = starbucks['gugunAll'] # gugunAll 컬렉션을 만든다.
gugunAll.delete_many({}) # gugunAll 컬렉션에 저장된 모든 데이터를 제거한다.
for g in gugunList:
    gugunAll.insert_one(g)

In [11]:
# gugunList = gugunAll.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1, 'gugun_cd': 1, 'gugun_nm': 1})
gugunList = gugunAll.find({}, {'_id': 0})
for s in gugunList:
    print(s)

{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0101', 'gugun_nm': '강남구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0120', 'gugun_nm': '강동구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0102', 'gugun_nm': '강북구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0103', 'gugun_nm': '강서구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0104', 'gugun_nm': '관악구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0105', 'gugun_nm': '광진구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0121', 'gugun_nm': '구로구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0106', 'gugun_nm': '금천구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0107', 'gugun_nm': '노원구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0108', 'gugun_nm': '도봉구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0122', 'gugun_nm': '동대문구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0109', 'gugun_nm': '동작구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0110', 'gugun_nm': '마포구'}
{'sido_cd': '01', 'sido_nm': '서울', 'g